## NSCP Section 407 One Way Slabs

In [34]:
import math

def check_oneway_twoway (longspan, shortspan):

    # center to center span per slab
    
    ratio = longspan / shortspan
    if ratio > 1.5:
        return "One-way slab"
    else:
        return "Two-way slab"
    
def slabthickness(clearspan, support, fy, wc, giventhickness):
    """
    Table 407.3.1.1 Minimum thickness of one-way slabs

    clearspan: clear span length (mm) >> shortspan - beam width
    support: simply, one-end, both-end, or cantilever
    fy: yield strength (MPa)
    wc: unit weight of concrete (kg/m³)
    giventhickness: proposed slab thickness (mm)

    returns: dictionary with the computed minimum thickness, final thickness, and a note about the selection
    """
    # Calculate minimum thickness based on support type
    if support == 'simply':
        minthk = clearspan / 20
    elif support == 'one-end':
        minthk = clearspan / 24
    elif support == 'both-end':
        minthk = clearspan / 28
    elif support == 'cantilever':
        minthk = clearspan / 10
    else:
        raise ValueError("Invalid support type; choose 'simply', 'one-end', 'both-end', or 'cantilever'")

    # Adjust thickness for fy (if not the typical 420 MPa)
    if fy != 420:
        minthk *= (0.4 + fy / 700)

    # Adjust for lightweight concrete (wc between 1440 and 1840)
    if 1440 <= wc <= 1840:
        multiplier = max(1.65 - 0.0003 * wc, 1.09)
        minthk *= multiplier

    # Compare the given thickness with the calculated minimum thickness
    if giventhickness < minthk:
        thickness = minthk
        note = "given thickness is less than minimum thickness, use minimum thickness"
    else:
        thickness = giventhickness
        note = "given thickness meets or exceeds the minimum thickness"

    return {
        "minimumthickness": minthk,
        "finalthickness": thickness,
        "note": note
    }

def rho (mu, phi_flexure, width, depth, fc, fy, beta1):
    Rn = mu / (phi_flexure * width * math.pow(depth,2))
    rho = 0.85 * fc / fy * (1 - math.sqrt(1 - 2*Rn/(0.85 * fc)))
    rhomax = 3/8 * 0.85 * fc * beta1 / fy
    if rho < rhomax:
        return "Okay"
    else:
        raise ValueError("Reinforcement ratio exceeds the maximum value")

def area_flexuralreinforcement (rho, width, depth, fy, thickness):
    """
    Table 407.6.1.1 As min for Non-Prestressed One-Way Slabs
    """
    # Calculated As
    ascalc = rho * width * depth

    # Minimum As
    areagross = width * thickness

    if fy < 420:
        asmin = 0.0020 * areagross
    else:
        asmin = max(0.0018 * 420 * areagross / fy, 0.0014 * areagross)

    # Adjust As
    if ascalc < asmin:
        mainbar_as = asmin
    else:
        mainbar_as = ascalc
    
    return {
        "mainbar_as": mainbar_as
    }

def spacing_flexuralreinforcement(mainbar_as, width, height, db, dagg):
    """
    Calculate Spacing by Width / Number of Bars,
    Where Number of Bars = As total / As of one bar
    """
    # Calculated Spacing
    bararea = math.pi * math.pow(db / 2, 2)
    calcspacing = width / (mainbar_as / bararea)

    # Minimum and Maximum spacing per code
    smin = max(25, db, 4/3 * dagg)  # mm
    smax = min(3 * height, 450)  # mm

    # Adjust spacing
    if calcspacing < smin:
        mainbarspacing = smin
    elif calcspacing > smax:
        mainbarspacing = smax
    else:
        mainbarspacing = calcspacing

    return {"mainbarspacing": mainbarspacing}

def temperaturebars (width, thickness, height, fy, dtemp):

    """424.4.3 Shrinkage and Temperature Reinforcement for Non-Prestressed Reinforcement"""

    areagross = width * thickness

    if fy < 420:
        ast_min = 0.0020 * areagross
    else:
        ast_min = max(0.0018 * 420 * areagross / fy, 0.0014 * areagross)

    """
    Calculate Spacing by Width / Number of Bars,
    Where Number of Bars = As total / As of one bar

    407.7.6.2 Maximum spacing of shrinkage and temperature reinforcement
    """

    calcspacing = width / (ast_min / (math.pi * math.pow(dtemp/2,2)))

    stmax = min (5*height, 450) #mm
    if calcspacing > stmax:
        tempspacing = stmax
    else:  
        tempspacing = calcspacing

    return {
        "ast_min": ast_min, 
        "tempspacing": tempspacing
    }

def shearreinforcement (vc, vu, phi, ds, fc, width, fyt, vs):
    """
    NSCP 2015 407.6.3 and Table 409.6.3.3: Minimum shear Reinforcement
    """
    phivc = phi * vc
    avmin = math.pi * math.pow (ds/2, 2)
    
    if vu > phivc:
        calcsmin = avmin / (max (0.062 * math.sqrt (fc) * width / fyt, 0.35 * width / fyt))
    else: 
        raise ValueError("Avmin is not required")

    """
    Table 409.7.6.2.2 Maximum Spacing of Shear Reinforcement
    """
    # Maximum Spacing
    if vs <= 1/3 * math.sqrt (fc) * width * depth:
	    smax = min (depth/2, 600)
    else:
	    smax = min (depth/4, 300)

    # Adjust Spacing
    if calcsmin > smax:
        shearspacing = smax
    else:
        shearspacing = calcsmin
        
    return {
        "avmin": avmin, 
        "shearspacing": shearspacing
    }

if __name__ == "__main__":
    # Sample usage for check_oneway_twoway:
    slab_type = check_oneway_twoway(6000, 3000)
    print("Slab type:", slab_type)
    
    # Sample usage for slabthickness:
    thickness_results = slabthickness(5000, "simply", 420, 1500, 250)
    print("Slab thickness results:", thickness_results)
    
    # Sample usage for rho:
    try:
        rho_status = rho(500, 0.9, 1000, 150, 30, 420, 0.85)
        print("Reinforcement ratio check:", rho_status)
    except ValueError as e:
        print("Error in reinforcement ratio check:", e)
    
    # Sample usage for area_flexuralreinforcement:
    # Using a sample reinforcement ratio (rho) value of 0.01
    area_results = area_flexuralreinforcement(0.01, 1000, 150, 420, 250)
    print("Flexural reinforcement area:", area_results)
    
    # Sample usage for spacing_flexuralreinforcement:
    spacing_results = spacing_flexuralreinforcement(area_results["mainbar_as"], 1000, 250, 16, 20)
    print("Flexural reinforcement spacing:", spacing_results)
    
    # Sample usage for temperaturebars:
    tempbar_results = temperaturebars(1000, 250, 250, 420, 16)
    print("Temperature bars:", tempbar_results)
    
    # Define a global depth variable for shearreinforcement
    depth = 250  # Slab depth in mm
    
    # Sample usage for shearreinforcement:
    try:
        shear_results = shearreinforcement(0.2, 0.3, 0.75, 16, 30, 1000, 500, 200)
        print("Shear reinforcement:", shear_results)
    except ValueError as e:
        print("Error in shear reinforcement calculation:", e)

Slab type: One-way slab
Slab thickness results: {'minimumthickness': 300.0, 'finalthickness': 300.0, 'note': 'given thickness is less than minimum thickness, use minimum thickness'}
Reinforcement ratio check: Okay
Flexural reinforcement area: {'mainbar_as': 1500.0}
Flexural reinforcement spacing: {'mainbarspacing': 134.0412865531645}
Temperature bars: {'ast_min': 450.0, 'tempspacing': 446.80428851054836}
Shear reinforcement: {'avmin': 201.06192982974676, 'shearspacing': 125.0}
